# BioImage Model Zoo localy and image analysis

In [ ]:
#for using GPU on Mac
#pip install tensorflow-macos tensorflow-metal

In [ ]:
import matplotlib.pyplot as plt
import zarr
import dask
import dask.array as da
import numpy as np
import bioimageio.core
from bioimageio.core import Tensor, Sample, create_prediction_pipeline
from bioimageio.spec.utils import load_array
from dask import delayed
import imageio
from tifffile import imread
from mpl_toolkits.mplot3d import Axes3D
import torch


import re
import os
import tifffile as tiff
from tifffile import imread
from skimage.filters import threshold_otsu
import time

In [2]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
tf.debugging.set_log_device_placement(True)

In [ ]:
#check if tensorflow works
a = tf.constant([[1.0, 2.0]])
b = tf.constant([[3.0], [4.0]])
c = tf.matmul(a, b)
print(c)

input: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
_EagerConst: (_EagerConst): /job:localhost/replica:0/task:0/device:GPU:0
output_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
a: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
b: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
MatMul: (MatMul): /job:localhost/replica:0/task:0/device:GPU:0
product_RetVal: (_Retval): /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor([[11.]], shape=(1, 1), dtype=float32)


2025-08-17 19:45:05.160876: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-08-17 19:45:05.161006: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-08-17 19:45:05.161026: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-08-17 19:45:05.161493: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-08-17 19:45:05.161524: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-08-17 19:45:05.192058: I tensorflow/core/common_runtime/placer.cc:125] input: (_Arg): /job:localhost/replica:0/task:0/device:GPU:0
2025-08-17 19:45:05.192233: I tensorflow/core/commo

In [5]:
#Function to interact with a 3D image
%matplotlib inline
from ipywidgets import *

def update(image, z=0):
    fig = plt.figure(figsize=(10, 10))
    plt.subplot(121)
    c = 0
    t = 0
    plt.imshow(image[t, c, z, :, :])
    fig.canvas.flush_events()

def update3ch(image, z=0):
    fig = plt.figure(figsize=(6, 6))
    rgb = np.stack([
        image[0, 1, z],  # channel 0 -> Green
        image[0, 0, z],  # channel 1 -> Red
        image[0, 2, z],  # channel 2 -> Blue
    ], axis=-1)

    #Values normalization, if they are not in [0,1]
    rgb = rgb.astype(np.float32)
    rgb -= rgb.min()
    rgb /= rgb.max()

    plt.imshow(rgb)
    plt.title(f"Z = {z}")
    plt.axis("off")
    plt.show()

def update3ch_together(image, z=0):
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    channel_names = ['Green=EdU', 'Red=gammaH2AX', 'Blue=53BP1']
    for c in range(3):
        axes[c].imshow(image[0, c, z, :, :], cmap='gray')
        axes[c].set_title(f'{channel_names[c]} channel - Z={z}')
        axes[c].axis('off')

    plt.tight_layout()
    plt.show()

#Display the results of masking (probabilities prediction)
def display(i=0):
    prediction, sample, inp_id, outp_id, name = results[i]
    pred_array = np.asarray(prediction.members[outp_id].data)
    fig = plt.figure(figsize=(10, 10))
    plt.subplot(121)
    plt.imshow(pred_array[0, 0, :, :])
    plt.title(name)
    fig.canvas.flush_events()

binarized_results = []

# Function to display and collect binarized results
def display_bin(i=0):
    prediction, sample, inp_id, outp_id, name = results[i]
    pred_array = np.asarray(prediction.members[outp_id].data)
    binary_pred = pred_array[0, 0, :, :] >= 0.5
    binarized_results.append(binary_pred)  # Append binarized 2D result (mask)

    fig = plt.figure(figsize=(10, 10))
    plt.subplot(121)
    plt.imshow(binary_pred[:, :])
    plt.title(name)
    fig.canvas.flush_events()


In [6]:
# Function to handle model inference
def run_model_inference(bmz_model, arr):
    # load model
    model_resource = bioimageio.core.load_description(bmz_model)

    # load model's test input image
    test_input_image = load_array(model_resource.inputs[0].test_tensor)

    # match test data type with the data type of the model input
    arr = arr.astype(test_input_image.dtype)

    # create input tensor
    input_tensor = Tensor.from_numpy(arr, dims=tuple(model_resource.inputs[0].axes))

    # create collection of tensors (sample)
    inp_id = model_resource.inputs[0].id
    outp_id = model_resource.outputs[0].id
    sample = Sample(members={inp_id: input_tensor}, stat={}, id="id")

    # The prediction_pipeline function is used to run a prediction with a given model
    # It applies the pre-processing, if indicated in the model rdf.yaml,
    # runs inference with the model and applies the post-processing, again if specified in the model rdf.yaml.
    prediction_pipeline = create_prediction_pipeline(model_resource)

    # Use the new prediction pipeline to run a prediction. The prediction pipeline returns a Sample object
    prediction = prediction_pipeline.predict_sample_without_blocking(sample)

    return prediction, sample, inp_id, outp_id

In [7]:
def create_a_mask(image, z):
    #plane = image_3D[z, :, :].compute() # convert the Dask array into a Numpy array
    prediction, sample, inp_id, outp_id = run_model_inference(bmz_model_id, image)
    name = "z:%s" % (z)
    return prediction, sample, inp_id, outp_id, name

## Import data and image processing

### Import

In [ ]:
#Our path
folder_path = ('./BDS3_2025_data/MXT') #path to your image

#Choose all the files in .tiff format
file_list = [f for f in os.listdir(folder_path) if f.endswith('.tif')]

In [9]:
%time
# Extract unique base names (without _C1, _C2, _C3 before .tif)
base_names = set()
for f in file_list:
    if '_C' in f:
        base_name = f.split('_C')[0]
        base_names.add(base_name)

#Sort base names for consistency:
base_names = sorted(list(base_names))

#Template for extracting cycle phase name from the file name:
pattern = re.compile(r'1h-1h_(.+?)_Pos') #with usage of RegEx packade (re)

#Data storage list: each item is [C1, C2, C3]
dict = {}

for name in base_names:
    match = pattern.search(name)
    if not match:
        continue
    phase = match.group(1)

    c1_path = os.path.join(folder_path, f"{name}_C1.tif")
    c2_path = os.path.join(folder_path, f"{name}_C2.tif")
    c3_path = os.path.join(folder_path, f"{name}_C3.tif")

    if all(os.path.exists(p) for p in [c1_path, c2_path, c3_path]):
        #add to dictionary
        if phase not in dict:
            dict[phase] = []
        dict[phase].append([c1_path, c2_path, c3_path])
    else:
        print(f"Lack of a channel for: {name}")
#results format will be like:
#{'G': [[c1_path, c2_path, c3_path], ...], 'SIII': [...], ...}

CPU times: user 3 μs, sys: 1e+03 ns, total: 4 μs
Wall time: 7.15 μs


In [10]:
print("Phases:")
for phase in dict:
    print(f"- {phase}")

Phases:
- G
- S_IV
- S_I
- S_V
- S_III
- S_II


In [ ]:
phases = list(dict.keys())
print(phases)

In [12]:
for phase in dict:
    count = len(dict[phase])
    print(f"Phase '{phase}' contains {count} samples(sample_index: 0)")

Phase 'G' contains 1 samples(sample_index: 0)
Phase 'S_IV' contains 1 samples(sample_index: 0)
Phase 'S_I' contains 1 samples(sample_index: 0)
Phase 'S_V' contains 1 samples(sample_index: 0)
Phase 'S_III' contains 1 samples(sample_index: 0)
Phase 'S_II' contains 1 samples(sample_index: 0)


In [13]:
#show first image -> [0], second channel - [1]
image_path = dict["S_III"][0][1] #dict[phase][sample_index][channel]
our_image = imread(image_path)
our_image.shape

(98, 512, 512)

In [ ]:
bmz_model_id = bioimageio.core.load_description('./affable-shark/rdf.yaml') #path to your model; you need to download the folder with model from website

#folder_name = "predictions"
#if not os.path.exists(folder_name):
    #os.makedirs(folder_name)
    #print(f"Folder '{folder_name}' created successfully!")
#else:
    #print(f"Folder '{folder_name}' already exists.")


folder_name = "masks"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Folder '{folder_name}' created successfully!")
else:
    print(f"Folder '{folder_name}' already exists.")

Folder 'masks1' already exists.


### Processing 

In [ ]:
def process_images(image_data):
    for image in image_data:
        img = imread(image)
        print(img.shape)

        #sample_ch = f.split(folder_path[0])
        #sample_ch = os.path.basename(image)
        sample_ch = os.path.splitext(os.path.basename(image))[0]
        print(sample_ch)
        #print (f"Image name: {sample_ch}")

        #save image as zarr
        zarr_path = "./zarr_files/" + sample_ch + ".zarr"
        #print(zarr_path)
        zarr.save(zarr_path, img)
        print("Saved as Zarr:", zarr_path)
        z = zarr.open(zarr_path, mode='r')
        z_dask = da.from_zarr(z)
        print(f"Z_dask shape: {z_dask.shape}")

        lazy_results = []

        for z in range(z_dask.shape[0]):
          r = delayed(create_a_mask)(z_dask[z, :, :] , z)
          lazy_results.append(r)

        print(time.ctime())
        results = dask.compute(*lazy_results)
        print(time.ctime())

        #save prediction
        #prediction_stack = np.stack(results, axis=0)
        #output_path = "./predictions/" + sample_ch + ".tif"
        #tiff.imwrite(output_path, prediction_stack.astype(np.uint8))

        #masks
        # List to store binarized planes
        binarized_results = []

          # Iterate over all planes
        for i in range(len(results)):
          prediction, sample, inp_id, outp_id, name = results[i]
          pred_array = np.asarray(prediction.members[outp_id].data)
          # Extract the 2D plane
          plane = pred_array[0, 0, :, :]
          # Binarize using Otsu’s method
          #threshold = threshold_otsu(plane)
          threshold = 0.5
          binary_plane = plane >= threshold  # True (1) for foreground, False (0) for background

          binarized_results.append(binary_plane.astype(np.uint8))  # Convert to uint8 (0 and 1)
          # Convert list to 3D array
        mask_3d = np.stack(binarized_results, axis=0)  # Stack along Z-axis
        drive_output_dir = "./masks1"
        os.makedirs(drive_output_dir, exist_ok=True)
        output_path = os.path.join(drive_output_dir, f"{sample_ch}.tiff")
        imageio.volsave(output_path, mask_3d.astype(np.uint8))

        mask_path = "./masks/" + sample_ch + ".tif"
        imageio.volsave(mask_path, mask_3d.astype(np.uint8))
        print("Saved binarized 3D mask.")

    return print("Finally!")

For one phase (three images)

In [ ]:
#to call the function, you need:
process_images(dict["G"][0])
#print(shape)

(97, 512, 512)
2015.01.17_MTX_1h-1h_G_Pos033_S001_0_C1
Saved as Zarr: ./zarr_files/2015.01.17_MTX_1h-1h_G_Pos033_S001_0_C1.zarr
Z_dask shape: (97, 512, 512)
Sun Aug 17 18:53:28 2025


2025-08-17 18:53:28.176 | WARNING  | bioimageio.spec._io:load_description:84 - returning already loaded description 'Nuclei Segmentation Boundary Model' as is
2025-08-17 18:53:28.190 | WARNING  | bioimageio.spec._io:load_description:84 - returning already loaded description 'Nuclei Segmentation Boundary Model' as is
2025-08-17 18:53:28.190 | WARNING  | bioimageio.spec._io:load_description:84 - returning already loaded description 'Nuclei Segmentation Boundary Model' as is
2025-08-17 18:53:28.191 | WARNING  | bioimageio.spec._io:load_description:84 - returning already loaded description 'Nuclei Segmentation Boundary Model' as is
2025-08-17 18:53:28.191 | WARNING  | bioimageio.spec._io:load_description:84 - returning already loaded description 'Nuclei Segmentation Boundary Model' as is
2025-08-17 18:53:28.192 | WARNING  | bioimageio.spec._io:load_description:84 - returning already loaded description 'Nuclei Segmentation Boundary Model' as is
2025-08-17 18:53:28.192 | WARNING  | bioimagei

KeyboardInterrupt: 

2025-08-17 18:56:31.762 | WARNING  | bioimageio.spec._io:load_description:84 - returning already loaded description 'Nuclei Segmentation Boundary Model' as is
2025-08-17 18:56:31.781 | WARNING  | bioimageio.spec._io:load_description:84 - returning already loaded description 'Nuclei Segmentation Boundary Model' as is
2025-08-17 18:56:31.781 | WARNING  | bioimageio.spec._io:load_description:84 - returning already loaded description 'Nuclei Segmentation Boundary Model' as is
2025-08-17 18:56:31.782 | WARNING  | bioimageio.spec._io:load_description:84 - returning already loaded description 'Nuclei Segmentation Boundary Model' as is
2025-08-17 18:56:31.782 | WARNING  | bioimageio.spec._io:load_description:84 - returning already loaded description 'Nuclei Segmentation Boundary Model' as is
2025-08-17 18:56:31.782 | WARNING  | bioimageio.spec._io:load_description:84 - returning already loaded description 'Nuclei Segmentation Boundary Model' as is
2025-08-17 18:56:31.783 | WARNING  | bioimagei

For all images from folder

In [ ]:
#without GPU
for phase in phases:
    process_images(dict[phase][0])
    print(f"Processing of phase {phase} completed")

print("Completed all.")


In [ ]:
#with GPU
for phase in phases:
    with tf.device('/GPU:0'): 
        process_images(dict[phase][0])
    print(f"Processing of phase {phase} completed")

print("Completed all.")


(97, 512, 512)
2015.01.17_MTX_1h-1h_G_Pos033_S001_0_C1
Saved as Zarr: ./zarr_files/2015.01.17_MTX_1h-1h_G_Pos033_S001_0_C1.zarr
Z_dask shape: (97, 512, 512)
Sun Aug 17 19:45:47 2025


2025-08-17 19:45:47.369 | WARNING  | bioimageio.spec._io:load_description:84 - returning already loaded description 'Nuclei Segmentation Boundary Model' as is
2025-08-17 19:45:47.371 | WARNING  | bioimageio.spec._io:load_description:84 - returning already loaded description 'Nuclei Segmentation Boundary Model' as is
2025-08-17 19:45:47.377 | WARNING  | bioimageio.spec._io:load_description:84 - returning already loaded description 'Nuclei Segmentation Boundary Model' as is
2025-08-17 19:45:47.378 | WARNING  | bioimageio.spec._io:load_description:84 - returning already loaded description 'Nuclei Segmentation Boundary Model' as is
2025-08-17 19:45:47.381 | WARNING  | bioimageio.spec._io:load_description:84 - returning already loaded description 'Nuclei Segmentation Boundary Model' as is
2025-08-17 19:45:47.381 | WARNING  | bioimageio.spec._io:load_description:84 - returning already loaded description 'Nuclei Segmentation Boundary Model' as is
2025-08-17 19:45:47.382 | WARNING  | bioimagei

KeyboardInterrupt: 